## Mount the drive. 
#### This applies to running the code on Google collab.

In [9]:
#Uncomment this to mount file system if you are running on Googl Colab
from google.colab import drive
drive.mount("/content/gdrive/")

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


## Load required packages

In [0]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
from keras.preprocessing.image import ImageDataGenerator,img_to_array
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
#from CNN import kcnn_6
import matplotlib.pyplot as plt
%matplotlib inline
from imutils import paths
import numpy as np
import random
import pickle
import cv2 as cv
import numpy as np
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras import backend as K
from keras.models import save_model,load_model
from collections import defaultdict
#import tqdm_utils
import os

In [0]:
#Comment this if not running on Colab
os.chdir("/content/gdrive/My Drive/Crop Disease")

## Initialize model parameters

In [0]:
# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
epochs = 5 #using small number of epochs because tje data available data is just for demonstration
#Full dataset is here: https://drive.google.com/drive/folders/1en8BwI3kyQFXJZh7R2vmCsp7WV9atpkM?usp=sharing
lr = 1e-3 #initial learning rate
bs = 32 #batch size
image_dims = (224,224 , 3) #Model input shape
# initialize the data and labels
data = []
labels = []
# grab the image paths and randomly shuffle them
#path to the train folder
imagePaths = sorted(list(paths.list_images("./train")))
random.seed(42)
random.shuffle(imagePaths)

In [0]:
# loop over the input images
for imagePath in imagePaths:
    
    # load the image, pre-process it, and store it in the data list
    image = cv.imread(imagePath)
    image = cv.resize(image, (image_dims[1], image_dims[0]),cv.INTER_AREA)
    image = img_to_array(image)
    data.append(image)

    # extract the class label from the image path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]

    labels.append(label)
  

In [21]:

# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
#printing the size of the image data. This value should be less that the RAM size of the
#computer otherwise you need to train the model on the generator
print("[INFO] data matrix: {:.2f}MB".format(
	data.nbytes / (1024 * 1000.0)))
# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.2)#, random_state=42)

[INFO] data matrix: 14.11MB


In [0]:
#remark: save the label binarizer to disk
f=open("./saved models/weights1.pickle","wb")
f.write(pickle.dumps(lb))
f.close()

In [0]:
#defining the base model. Transfer learning of InceptionV3

def base_model(use_imagenet=True):
    
    # load pre-trained model graph, don't add final layer
    model = (keras.applications.InceptionV3(include_top=False, input_shape=image_dims,
                                            weights='imagenet' if use_imagenet else None))
    
    # add global pooling just like in InceptionV3
    new_output = keras.layers.GlobalAveragePooling2D()(model.output)
    
    # add new dense layer for our labels
    new_output = keras.layers.Dense(len(lb.classes_), activation='softmax')(new_output)
    model = keras.engine.training.Model(model.inputs, new_output)
    
    return model

In [0]:
# initialize the model
model = base_model();

In [0]:
# set all layers trainable by default
for layer in model.layers:
    layer.trainable = True
    if isinstance(layer, keras.layers.BatchNormalization):
        # we do aggressive exponential smoothing of batch norm
        # parameters to faster adjust to our new dataset
        layer.momentum = 0.7

In [0]:
# fix deep layers (fine-tuning only last 50)
for layer in model.layers[:-50]:
    # fix all but batch norm layers, because we need to update moving averages for a new dataset!
    if not isinstance(layer, keras.layers.BatchNormalization):
        layer.trainable = False

In [0]:
#Define model checkpoints
filepath="./saved models/saved-model-{epoch:02d}-{val_loss:.2f}.hdf5"
#filepath = "saved-model-{epoch:02d}-{val_acc:.2f}.hdf5"
mc =  ModelCheckpoint(filepath,monitor="val_loss",verbose=1,save_best_only=True,mode="min")
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=9)
callbacks_list=[mc,es]

In [0]:
#remark: compile new model
model.compile(
    loss='categorical_crossentropy',  # we train 102-way classification
    optimizer=keras.optimizers.adamax(lr=1e-3),  # we can take big lr here because we fixed first layers
    metrics=["accuracy","mse"]  # report accuracy during training
)

In [32]:
#[INFO] training network..."
H = model.fit(
	trainX, trainY, batch_size=bs,
	validation_data=(testX, testY),
	epochs=epochs, verbose=1,callbacks=callbacks_list)#,steps_per_epoch=len(trainX) // BS

[INFO] training network...
Train on 9 samples, validate on 3 samples
Epoch 1/5
9/9 [==============================] - 19s 2s/step - loss: 1.3328 - accuracy: 0.3333 - mse: 0.2633 - val_loss: 0.3587 - val_accuracy: 1.0000 - val_mse: 0.0597

Epoch 00001: val_loss improved from inf to 0.35875, saving model to ./saved models/saved-model-01-0.36.hdf5
Epoch 2/5
9/9 [==============================] - 4s 396ms/step - loss: 4.6526e-04 - accuracy: 1.0000 - mse: 2.2514e-07 - val_loss: 0.2907 - val_accuracy: 1.0000 - val_mse: 0.0530

Epoch 00002: val_loss improved from 0.35875 to 0.29071, saving model to ./saved models/saved-model-02-0.29.hdf5
Epoch 3/5
9/9 [==============================] - 4s 399ms/step - loss: 1.2604e-04 - accuracy: 1.0000 - mse: 1.8944e-08 - val_loss: 0.2745 - val_accuracy: 1.0000 - val_mse: 0.0524

Epoch 00003: val_loss improved from 0.29071 to 0.27450, saving model to ./saved models/saved-model-03-0.27.hdf5
Epoch 4/5
9/9 [==============================] - 6s 676ms/step - loss